In [1]:
def composition(k, text):
    """
    Assuming k is an integer and text is a string of nucleotides, return the list of all k-mers in text (including repeated k-mers).
    >>> composition(5, "CAATCCAAC")
    ['CAATC', 'AATCC', 'ATCCA', 'TCCAA', 'CCAAC']
    """
    kmers = []
    length = len(text)
    for i in range(0, length-k+1):
        kmers.append(text[i:i+k])
    #print(*kmers, sep=" ")
    return kmers

In [2]:
def PathToGenome(path):
    """
    Assuming the input is a sequence path of n k-mers split with spaces, the consecutive ones of which shares an overlap whose length is k-1;
    return an assembled string.
    >>> PathToGenome("ACCGA CCGAA CGAAG GAAGC AAGCT")
    ACCGAAGCT
    """
    kmers = path.split() if type(path) == str else path
    return kmers[0] + ''.join(kmer[-1] for kmer in kmers[1:])

In [3]:
def Overlap(patterns):
    """
    Assuming the input is a string of k-mers split with spaces, return the overlap graph in the form of an adjacency list
    >>> Overlap("ATGCG GCATG CATGC AGGCA GGCAT GGCAC")
    {'GCATG': ['CATGC'],
    'CATGC': ['ATGCG'],
    'AGGCA': ['GGCAT', 'GGCAC'],
    'GGCAT': ['GCATG']}
    """
    kmers = patterns.split()
    prefix = {kmer: kmer[:-1] for kmer in kmers}
    suffix = {kmer: kmer[1:] for kmer in kmers}

    adjacency = {kmer: [] for kmer in kmers}
    for kmer in kmers:
        for key, value in prefix.items():
            if suffix[kmer] == value:
                adjacency[kmer].append(key)
    
    # Remove all the keys in adjacency if its value is empty list
    adjacency = {key: value for key, value in adjacency.items() if value}
    # print('\n'.join(f'{kmer}: {" ".join(adj)}' for kmer, adj in adjacency.items()))
    return adjacency

In [4]:
def DeBruijn_text(k, text):
    """
    Assuming k is an integer greater than 0 and text is a string whose length is greater than or equal to k,
    the output is an adjacency list where all the identically labelled nodes are glued.
    >>> DeBruijn_text(4, "AAGATTCTCTAAGA")
    {'AAG': ['AGA', 'AGA'],
    'AGA': ['GAT'],
    'GAT': ['ATT'],
    'ATT': ['TTC'],
    'TTC': ['TCT'],
    'TCT': ['CTC', 'CTA'],
    'CTC': ['TCT'],
    'CTA': ['TAA'],
    'TAA': ['AAG']}
    """
    length = len(text)
    kmers = [text[i:i+k-1] for i in range(0, length-k+2)]
    adjacency = {kmer: [] for kmer in kmers}
    for i in range(0, len(kmers)-1):
        adjacency[kmers[i]].append(kmers[i+1])
    
    # Remove all the keys in adjacency if its value is empty list
    adjacency = {key: value for key, value in adjacency.items() if value}

    #print('\n'.join(f'{kmer}: {" ".join(adj)}' for kmer, adj in adjacency.items()))
    return adjacency

In [5]:
def DeBruijn_pattern(patterns):
    """
    Assume the input is a collection of k-mer patterns split with spaces,
    the output is an adjacency list of the de Bruijn graph.
    >>> DeBruijn_pattern("GAGG CAGG GGGG GGGA CAGG AGGG GGAG")
    {'GAG': ['AGG'],
    'CAG': ['AGG', 'AGG'],
    'GGG': ['GGG', 'GGA'],
    'AGG': ['GGG'],
    'GGA': ['GAG']}
    """
    patterns = patterns.split()
    k = len(patterns[0])
    adjacency = {kmer[:k-1]: [] for kmer in patterns}

    for kmer in patterns:
        adjacency[kmer[:k-1]].append(kmer[1:])
    
    #print('\n'.join(f'{kmer}: {" ".join(adj)}' for kmer, adj in adjacency.items()))
    return adjacency

In [6]:
def EulerianCycle(graph):
    """
    Assume the input is a strongly connected directed graph in the form of an adjacency list, the output is an Eulerian cycle.
    >>> EulerianCycle(
    '''
    0: 3
    1: 0
    2: 1 6
    3: 2
    4: 2
    5: 4
    6: 5 8
    7: 9
    8: 7
    9: 6
    ''')
    ['2', '6', '8', '7', '9', '6', '5', '4', '2', '1', '0', '3', '2']
    """
    import random

    # Re-format the input and generate a graph dictionary where the key is node and value is a list consisting of its adjacent nodes, e.g., graph = {"a": ["b", "c"]}
    if type(graph) == str:
        node_graph = [i for i in [item.strip() for item in graph.split("\n")] if i != ""]
        graph = {str(key): list(map(str, value.split())) for key, value in (item.split(": ") for item in node_graph)}

    start = random.choice(list(graph.keys()))
    cycle = [start]
    while graph != {}:
        if cycle[0] == cycle[-1] and cycle[-1] not in graph:
            new_start = random.choice([node for node in cycle if node in graph.keys()])
            cycle = cycle[cycle.index(new_start):] + cycle[1:cycle.index(new_start)+1]
        else:
            next_step = random.choice(graph[cycle[-1]])
            graph[cycle[-1]].remove(next_step)
            cycle.append(next_step)
            graph = {key: value for key, value in graph.items() if value}
    #print(" ".join(str(x) for x in cycle))
    return cycle

In [26]:
def EulerianPath(graph):
    """
    Assume the input is a nearly balanced directed graph, the output is its Eulerian path.
    >>> EulerianPath(
    '''
    0: 2
    1: 3
    2: 1
    3: 0 4
    6: 3 7
    7: 8
    8: 9
    9: 6
    ''')
    ['6', '7', '8', '9', '6', '3', '0', '2', '1', '3', '4']
    """
    import random
    from collections import Counter

    # Re-format the input and generate a graph dictionary where the key is node and value is a list consisting of its adjacent nodes, e.g., graph = {"a": ["b", "c"]}
    if type(graph) == str:
        node_graph = [i for i in [item.strip() for item in graph.split("\n")] if i != ""]
        graph = {str(key): list(map(str, value.split())) for key, value in (item.split(": ") for item in node_graph)}
    nodes = list(set([node for node in graph.keys()] + sum(graph.values(), [])))

    # Find the imbalanced nodes, and make them the start and the end of the Eulerian path
    for node in nodes:
        if node not in graph:
            graph[node] = []
        indegree = dict(Counter(sum(graph.values(), [])))[node] if node in dict(Counter(sum(graph.values(), []))) else 0
        outdegree = len(graph[node]) if graph[node] != [] else 0
        if indegree < outdegree:
            start = node
        elif indegree > outdegree:
            end = node

    # Append one directed edge from the end to the start, and generate an Eulerian cycle
    graph[end].append(start)
    cycle = [start]
    while graph != {}:
        if cycle[0] == cycle[-1] and cycle[-1] not in graph:
            new_start = random.choice([node for node in cycle if node in graph.keys()])
            cycle = cycle[cycle.index(new_start):] + cycle[1:cycle.index(new_start)+1]
        else:
            next_step = random.choice(graph[cycle[-1]])
            graph[cycle[-1]].remove(next_step)
            cycle.append(next_step)
            graph = {key: value for key, value in graph.items() if value}
    
    # Reshuffle the Eulerian cycle, remove the additional edge, and set the start and end
    path = cycle[cycle.index(end)+1:] + cycle[1:cycle.index(end)+1]
    while path[0] != start or path[-1] != end:
        path = path[path.index(end)+1:] + path[:path.index(end)+1]
    #print(" ".join(str(x) for x in path))
    return path

In [8]:
def StringReconstruction(k, patterns):
    """
    Assume the input is an integer k followed by a list of k-mers patterns,
    the output is a string with k-mer composition equal to patterns.
    >>> StringReconstruction(4, "CTTA ACCA TACC GGCT GCTT TTAC")
    GGCTTACCA
    """
    dB = DeBruijn_pattern(patterns)
    path = EulerianPath(dB)
    Text = PathToGenome(path)
    return Text

In [9]:
def BinaryString(k):
    """
    Assume the input is an integer k > 0, the output is a list colletion of all possible binary k-mers
    >>> BinaryString(3)
    ['000', '001', '010', '011', '100', '101', '110', '111']
    """
    if k == 1:
        return ["0", "1"]
    else:
        return [kmer + bit for kmer in BinaryString(k-1) for bit in ["0", "1"]]

def CircularString(k):
    """
    Assume the input is an integer k > 0, the output is a list of unique k-universal circular string
    and the total number of all unique circular strings.
    >>> CircularString(3)
    (['10001110', '00010111'], 2)
    """
    # Generate a binary k-mer list that includes all the possibilities
    binary_kmer = BinaryString(k)
    cycles = []
    for i in range(1000):
        path = EulerianCycle(DeBruijn_pattern(" ".join(binary_kmer)))
        Text = PathToGenome(path)
        cycles.append(Text[:-k+1]) # To remove the overlap whose length is k-1
    
    # Make a list composed of unique k-universal circular strings
    cycles = list(set(cycles)) # Make every element in the cycles unique
    sorted_substrings = {}

    for cycle in cycles:
        double_cycle = cycle + cycle # Double the cycle to make a linearized string
        substrings = [double_cycle[i:i+len(cycle)] for i in range(len(cycle))] # Create a list which includes all the k-mers whose length is the length of cycle
        sorted_substrings[" ".join(sorted(substrings))] = cycle # Discard all the strings that have the same k-mers

    return list(sorted_substrings.values()), len(sorted_substrings)

In [10]:
def PairedComposition(k, d, Text):
    """
    Assume the input k, d are integers, k > 0, d >= 0, and Text is a string whose length is >= 2k + d,
    the output is a list consisting of all pattern combinations in the form of (Pattern1|Pattern2),
    the length of pattern is k and their distance inbetween is d
    >>> PairedComposition(3, 2, "TAATGCCATGGGATGTT")
    ['(AAT|CAT)',
    '(ATG|ATG)',
    '(ATG|ATG)',
    '(CAT|GAT)',
    '(CCA|GGA)',
    '(GCC|GGG)',
    '(GGG|GTT)',
    '(TAA|CCA)',
    '(TGC|TGG)',
    '(TGG|TGT)']
    """
    return sorted(["(" + Text[i:i+k] + "|" + Text[i+k+d:i+d+2*k] + ")" for i in range(len(Text)-(2*k)-d+1)])

PairedComposition(3, 2, "TAATGCCATGGGATGTT")

In [27]:
def StringSpelledByGappedPatterns(k, d, path):
    """
    Assume the inputs are integers k and d, and an Eulerian path whose nodes are combinations of pre- and suffixes,
    the output is a collapsed single string.
    >>> StringSpelledByGappedPatterns(4, 2, "GACC|GCGC ACCG|CGCC CCGA|GCCG CGAG|CCGG GAGC|CGGA")
    'GACCGAGCGCCGGA'
    """
    path = path.split() if type(path) == str else path
    first_string, second_string = PathToGenome([node[:k] for node in path]), PathToGenome([node[-k:] for node in path])
    symbol = first_string[k+d:]
    return "There is no string spelled by the gapped patterns" if symbol != second_string[:len(symbol)] else first_string[:k+d] + second_string

def PairedCompositionGraph(k, d, collection):
    """
    Assume the input k, d are integers, k > 0, d >= 0, and collection is a set of paired k-mers,
    the output is a string with (k, d)-mer composition equal to collection
    >>> PairedCompositionGraph(4, 2, "GAGA|TTGA TCGT|GATG CGTG|ATGT TGGT|TGAG GTGA|TGTT GTGG|GTGA TGAG|GTTG GGTC|GAGA GTCG|AGAT")
    GTGGTCGTGAGATGTTGA
    """
    edges = [comb.split("|") for comb in collection.split(" ")] if type(collection) == str else collection
    nodes = [] # All the possible pre- and suffixes
    for edge in edges:
        nodes.append([edge[i][:-1] for i in [0, 1]]) # All the grouped prefixes
        nodes.append([edge[i][1:] for i in [0, 1]]) # All the grouped suffixes
    
    # Generate a De Bruijn adjacency list for all the nodes
    DeBruijn = {"|".join(node): [] for node in nodes}
    for edge in edges:
        DeBruijn["|".join([edge[i][:-1] for i in [0, 1]])].append("|".join([edge[i][1:] for i in [0, 1]]))

    Eulerian = EulerianPath(DeBruijn)
    
    return StringSpelledByGappedPatterns(k-1, d+1, Eulerian) # Because in the Eulerian path the nodes are 1-nt shorter than the edge

with open('dataset_30188_16.txt') as f:
    lines = f.read()
#print(lines)

PairedCompositionGraph(50, 200, lines)

KeyboardInterrupt: 